In [14]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the LSTM parameters
n_h = 2  # number of hidden units
n_x = 2  # input dimension
n_c = 2  # CEC memory units (same as n_h)

# Insert your weight matrices here
# W_f, W_i, W_c, W_o are numpy arrays of shape (n_h, n_h + n_x)
# For example:
# W_f = np.array([[Wfc11, Wfc12, Wfh11, Wfh12, Wfx11, Wfx12],
#                 [Wfc21, Wfc22, Wfh21, Wfh22, Wfx21, Wfx22]])
# Similarly for W_i, W_c, W_o
W_f = np.array([[-0.8,0,-0.3,-0.75,0.26,0.09],[0.05,-0.1,-0.9,0,0.85,0.07]])
# Replace the placeholders with your actual weight values
W_i = W_f

W_c = np.array([[0.8,0.9,0.3,1],[0.3,0.45,0.7,0.5]])

W_o = W_f

# Initial input x_t at time t (insert your input vector here)
# x_t is a numpy array of shape (n_x, 1)
x_t = np.array([[-1,1]]).T


# Initial input x_t at time t (insert your input vector here)
# x_t is a numpy array of shape (n_x, 1)
x_t = np.array([
    [-1.0],
    [1.0]
])

# Initialize c_prev and h_prev
c_prev = np.zeros((n_c, 1))
h_prev = np.zeros((n_h, 1))

# Record the initial norm of the cell state (will be updated after initial step)
norm_c_t_initial = None

# Set the threshold to 1/100th of the initial norm (to be set after initial step)
threshold = None

# Initialize variables for the loop
norm_c_t = None
N = 0  # Time steps counter

# Start the simulation
while True:
    # Concatenate c_prev, h_prev, and x_t appropriately
    concat_fio = np.vstack((c_prev, h_prev, x_t))  # Shape: (n_c + n_h + n_x, 1)
    concat_c = np.vstack((h_prev, x_t))            # Shape: (n_h + n_x, 1)

    # Compute gate activations
    f_t = sigmoid(np.dot(W_f, concat_fio))
    i_t = sigmoid(np.dot(W_i, concat_fio))
    o_t = sigmoid(np.dot(W_o, concat_fio))
    c_tilde_t = np.tanh(np.dot(W_c, concat_c))

    # Update cell state and hidden state
    c_t = f_t * c_prev + i_t * c_tilde_t
    h_t = o_t * np.tanh(c_t)

    # On the first iteration, record the initial norm
    if N == 0:
        norm_c_t_initial = np.linalg.norm(c_t)
        threshold = norm_c_t_initial / 100.0
        norm_c_t = norm_c_t_initial
    else:
        # Update the norm of the cell state
        norm_c_t = np.linalg.norm(c_t)

        # Check if the norm has fallen below the threshold
        if norm_c_t < threshold:
            break

    # Prepare for next time step
    N += 1
    c_prev = c_t
    h_prev = h_t

    # For t > initial time, the input is zero
    x_t = np.zeros((n_x, 1))

# Output the memory duration
print("Memory duration N =", N)


Memory duration N = 15


In [15]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Number of neurons in hidden layer and gates
n_h = 3  # hidden state dimension
n_x = 2  # input dimension

# Insert your weight matrices here
# W_z, W_r, W are numpy arrays of shape (n_h, n_h + n_x)
# W_z and W_r are for the update and reset gates, respectively
# W is for the candidate hidden state

# Example placeholder values (replace these with your actual weights)
W_z = np.array([
    # Whz1, Whz2, Whz3, Wxz1, Wxz2
    [0.7,0.8,0.3,0.8,0.7],
    [0.6,0.5,0.3,0.9,0.2],
    [0.9,0.1,0.2,0.3,0.3]
])

W_r = np.array([
    # Whr1, Whr2, Whr3, Wxr1, Wxr2
    [0.1,0.2,0.3,1.0,0.2],
    [0.6,0.5,0.4,0.5,1.0],
    [-0.1,0.2,0.3,0.9,0.1]
])

W = np.array([
    # Wh1, Wh2, Wh3, Wx1, Wx2
    [0.8,0.9,0.6,1.0,0.7],
    [0.1,0.2,0.3,0.8,1.0],
    [-0.1,0.5,0.8,1.0,0.4]
])

# Initial input x_t at time t (insert your input vector here)
x_t = np.array([
    [1],
    [1],
])

# Initialize h_prev
h_prev = np.zeros((n_h, 1))

# Record the initial norm of the hidden state
norm_h_t_initial = None

# Set the threshold to 1/100th of the initial norm (to be set after initial step)
threshold = None

# Initialize variables for the loop
norm_h_t = None
N = 0  # Time steps counter

# Start the simulation
while True:
    # Concatenate h_prev and x_t
    concat_hx = np.vstack((h_prev, x_t))  # Shape: (n_h + n_x, 1)
    
    # Compute gates
    z_t = sigmoid(np.dot(W_z, concat_hx))  # Update gate
    r_t = sigmoid(np.dot(W_r, concat_hx))  # Reset gate
    
    # Compute candidate hidden state
    r_h_prev = r_t * h_prev  # Element-wise multiplication
    concat_rh_x = np.vstack((r_h_prev, x_t))  # Shape: (n_h + n_x, 1)
    h_tilde = np.tanh(np.dot(W, concat_rh_x))
    
    # Update hidden state
    h_t = (1 - z_t) * h_prev + z_t * h_tilde
    
    # On the first iteration, record the initial norm
    if N == 0:
        norm_h_t_initial = np.linalg.norm(h_t)
        threshold = norm_h_t_initial / 100.0
        norm_h_t = norm_h_t_initial
    else:
        # Update the norm of the hidden state
        norm_h_t = np.linalg.norm(h_t)
        
        # Check if the norm has fallen below the threshold
        if norm_h_t < threshold:
            break
    
    # Prepare for next time step
    N += 1
    h_prev = h_t
    
    # For t > initial time, the input is zero
    x_t = np.zeros((n_x, 1))

# Output the memory duration
print("Memory duration N =", N)


Memory duration N = 18


In [10]:
import numpy as np
def get_posterior_probs(alpha, beta):
        """Compute posterior probabilities.

        Input
        -----
        alpha: (np.array, dim = (input_len, 2 * target_len + 1))
                forward probability

        beta: (np.array, dim = (input_len, 2 * target_len + 1))
                backward probability

        Return
        ------
        gamma: (np.array, dim = (input_len, 2 * target_len + 1))
                posterior probability

        """

        gamma = alpha*beta
        gamma /= np.sum(gamma)
        return gamma
alpha= np.array([0.09,0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01])
beta = np.flip(alpha)
gamma = get_posterior_probs(alpha,beta)
gamma_sum_=0
for i in [0,2,4,6,8]:
        gamma_sum_+= gamma[i]
print(gamma_sum_)

0.5151515151515152
